# Import 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy


---

## Thu thập dữ liệu
Dữ liệu được thu thập trên kaggle.
Link dữ liệu : https://www.kaggle.com/vipin20/loan-application-data  
Đây là tập dữ liệu thông tin về các khoản vay bao gồm thông tin cá nhân của người vay và khoản vay có được chấp nhận hay không.  
File dữ liệu bao gôm :  
- File `df1_loan.csv` : tập dữ liệu 

# Khám phá dữ liệu

## Đọc dữ liệu

![](https://i.pinimg.com/originals/e0/2f/31/e02f319e566acc11cbff272c16a650e1.gif)

In [ ]:
data_df = pd.read_csv("Data/df1_loan.csv")
data_df.head()

In [ ]:
data_df.info()

## Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [ ]:
data_df.shape

## Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Quan sát sơ bộ, mỗi dòng chứa thông tin của một cá nhân muốn vay và thông tin về việc khoản vay có được chấp nhận hay không

## Dữ liệu có các dòng bị lặp không?

In [ ]:
data_df.index.duplicated().sum()

## Mỗi cột có ý nghĩa gì?

In [ ]:
with open('description.txt', 'r') as f:
    print(f.read())

## Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

In [ ]:
col_dtypes = data_df.dtypes
col_dtypes

## Tiền xử lý

### Cột định danh

Có tới 2 cột định danh là cột đầu (chưa được đặt tên) và cột `Loan_ID`.  
Ta sẽ lựa chọn 1 trong 2 cột làm định danh và loại bỏ cột còn lại.  
Ở đây ta sẽ chọn chọn cột đầu (vì nó tốn ít bộ nhớ hơn so với cột `Loan_ID`)

In [ ]:
#data_df.drop(columns=['Loan_ID'])
try:
    data_df = data_df.rename({"Unnamed: 0":"Index"},axis=1).set_index("Index").drop(columns=['Loan_ID'])
except:
    pass
data_df = data_df.sort_index(axis=1)
data_df.head(7)

### Chuyển dtype của cột Toltal_income sang dạng số

Có thể thấy cột `Total_income` thực chât là biểu diễn số liệu nhưng lại được biểu diễn dưới dạng object do có kí tự "\$" đứng trước.  
Ta sẽ tiến hành loại bỏ kí tự "\$" đứng đầu và chuyển về dạng số.  

In [ ]:
try:
    data_df.Total_Income = data_df.Total_Income.str[1:].astype(np.float64)
except:
    pass

### Khám phá mới ! `Total_income = ApplicantIncome + CoapplicantIncome`

Theo quan sát, ta thấy được `Total_income = ApplicantIncome + CoapplicantIncome`.  
Kiểm chứng bằng code :

In [ ]:
np.all(data_df.ApplicantIncome + data_df.CoapplicantIncome == data_df.Total_Income)

Tuy nhiên, em quyết định không loại bỏ hai cột này vì từ hai cột có thể suy ra cột total nhưng từ cột total thì không thể suy ngược lại được. Vì vậy, em quyết định bỏ cột `Total_income` :-)

In [ ]:
try:
    data_df = data_df.drop(columns=["Total_Income"])
except:
    pass
data_df.head()

# Khám phá dữ liệu (tiếp tục)

## Cột `Dependents`
Là dữ liệu số nhưng phương thức `info()` lại cho ra là object. Ta hãy cùng tìm hiểu vấn đề này

In [ ]:
data_df.Dependents.value_counts()

Hóa ra do có dự liệu ghi dưới dạng "3+". Ta tiến hành xử lý chuyển 3+ thành 3

In [ ]:
try:
    data_df.Dependents = data_df.Dependents.str.replace("+","",regex=False).apply(pd.to_numeric)
except:
    pass

## Phân tích sự phân bố giá trị của các cột

### Xác định các loại cột

Ta sẽ cho cột numeric là các cột có nhiều hơn 10 giá trị khác nhau. Các cột còn lại là category.

In [ ]:
cate_cols = data_df.columns[data_df.nunique() <= 9]
nume_cols = data_df.columns[data_df.nunique() > 10]

In [ ]:
data_df['Loan_Status'].value_counts().plot.bar()

### Dữ liệu categories

In [ ]:
for col in cate_cols:
    data_df[col].value_counts().plot.bar()
    plt.show()
    plt.xlabel(col)


In [ ]:
for col in nume_cols:
    data_df[col].plot.hist(bins=50)
    plt.xlabel(col)
    plt.show()

- Ta nhận thấy outlier của các **cột dữ liệu numerical** khá là kinh dị và sẽ rất tệ nếu ta không xử lý chúng.
- Cột `Loan_amount_term` cũng cần phải xử lý 

# Tiếp tục tiền xử lý

## Xóa outlier như thế nào?

- Quan sát biểu đồ được trực quan hóa, ta có thể thấy giá trị của cột cột có dạng phân phối chuẩn (theo kinh nghiệm nhìn biểu đồ nhiều).
- Theo định luật Empirical : https://www.investopedia.com/terms/e/empirical-rule.asp
- 99.7% dữ liệu nằm trong khoảng [mean - 3*std , mean + 3 *std], những phần còn lại nằm ngoài khoảng này đều là linh tinh hết (outlier).

In [ ]:
def notOutlierRange(s):
    _mean = s.mean()
    _std = s.std()
    return (_mean - 3 * _std , _mean + 3* _std)

In [ ]:
notOutlierRange(data_df.Loan_Amount_Term)

## Đợi đã, việc xóa này có ảnh hưởng đến tính trực quan của dữ liệu không?

Đầu tiên, ta sẽ xem số outlier bị loại bỏ có nhiều không bằng cách tính tỉ lệ outlier có trong từng cột

In [ ]:

for col in nume_cols:
    front,rear = notOutlierRange(data_df[col])
    s = data_df[col]
    before = len(s)
    #print(
    after = len(s.loc[(s > front) & (s < rear)])
    print(col)
    print("{}%".format(((before-after)/after)*100))


Hai cột trên chỉ có 1.4 %, không đáng quan ngại. Nhưng LoanAmount lên tới 6%!  
Hãy cùng xem tìm chuyện gì sẽ xảy ra nếu ta loại bỏ outlier.

In [ ]:
data_df_copy = copy.copy(data_df)
for col in nume_cols:
    front,rear = notOutlierRange(data_df[col])
    print(front,rear)
    s = data_df[col]
    s.loc[(s > front) & (s < rear)].plot.hist(bins=50)
    plt.xlabel(col)
    plt.show()

In [ ]:
for col in nume_cols:
    front,rear = notOutlierRange(data_df[col])
    s = data_df[col]
    s.loc[(s > front) & (s < rear)].plot.hist(bins=50)
    plt.xlabel(col)
    plt.show()